In [ ]:
import importlib
import numpy as np
import matplotlib.pyplot as plt

import colors, plot_func, helpers, file_helpers, experiments

## Supervisor

In [ ]:
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

Sprvsr = SupervisorClass()
# Sprvsr.init_dataset()

## Sensor

In [ ]:
importlib.reload(file_helpers)
importlib.reload(plot_func)

import ForsentekClass
importlib.reload(ForsentekClass)
from ForsentekClass import ForsentekClass

Snsr = ForsentekClass()

## robot

In [ ]:
if "m" in globals():
    try:
        m.disconnect()
    except Exception as e:
        print("Robot was not connected or already disconnected:", e)
    del m


import MecaClass
importlib.reload(MecaClass)
importlib.reload(file_helpers)
importlib.reload(helpers)
from MecaClass import MecaClass

m = MecaClass(Sprvsr)

In [ ]:
m.connect()
# m.move_to_origin()
m._get_current_pos()
print(f'current pos {m.current_pos}')
print(f'relative to origin {m.current_pos[:2]-m.pos_origin}')

## Calibrate

In [ ]:
# Without chain you can use False
# If with chain, use True

Snsr.calibrate_daily(V0_from_file = True)

## Training

In [ ]:
m.set_TRF_wrt_holder(mod='stress_strain')
m.robot.MoveLin(129.5, -12.5, 27, -180, 0, 0)  # down
# m.robot.MoveLin(130, -14, 41, -180, 0, 0)  # up

In [ ]:
importlib.reload(experiments)

theta_range = 154
N = 20

# thetas_vec, Fx_vec, Fy_vec = experiments.stress_strain(m, Snsr, 
#                                                        theta_range, N,
#                                                        connect_hinge = False)

# ------ Experiment prelims ------
init_pos_6 = m.robot.GetPose()
thetas_vec_up = np.linspace(-theta_range, theta_range, int(N/2))
thetas_vec_down = np.linspace(theta_range, -theta_range, int(N/2))
if N % 2 == 1:
   thetas_vec_down = np.append(thetas_vec_down, -theta_range) 
thetas_vec = np.append(thetas_vec_up, thetas_vec_down)
Fx_vec = np.zeros((N,))
Fy_vec = np.zeros((N,))

# ------ Experiment ------
for i, theta in enumerate(thetas_vec):
    # move
    pos_6 = (init_pos_6[0], init_pos_6[1], init_pos_6[2], init_pos_6[3], init_pos_6[4], theta)
    print(f'pos_6{pos_6}')
    # m.robot.MoveLine(*pos_6)
    m._get_current_pos()
    print('current_pos=', m.current_pos)
    m.move_lin_or_pose(pos_6, mod='lin')

    # measure
    Snsr.measure()
    Fx, Fy = np.mean(Snsr.force_data[:, 0]), np.mean(Snsr.force_data[:, 1])

    # save
    Fx_vec[i], Fy_vec[i] = Fx, Fy

# ------- de-stress at end ------
end_pos = (init_pos_6[0], init_pos_6[1], init_pos_6[2], init_pos_6[3], init_pos_6[4], 0.0)
m.robot.MoveLin(*end_pos)

In [ ]:
scale = 1000  # mili
arm_len = 0.0225

torque_x = Fx_vec * arm_len * scale
torque_y = Fy_vec * arm_len * scale

plt.plot(np.deg2rad(thetas_vec), torque_x)
plt.plot(np.deg2rad(thetas_vec), torque_y)
plt.xlabel(r'$\theta\,\left[rad\right]$')
plt.legend([r'$\tau_x$', r'$\tau_y$'])
plt.ylim([-85, 85])
plt.show()

In [ ]:
plt.plot(np.deg2rad(thetas_vec), Fx_vec)
plt.plot(np.deg2rad(thetas_vec), Fy_vec)
plt.xlabel(r'$\theta\,\left[rad\right]$')
plt.legend([r'$F_x$', r'$F_y$'])
plt.ylim([-5, 5])
plt.show()

In [ ]:
## Load

Sprvsr.init_dataset(dataset_path = "data/datasets/Jan29/buckle[0111]minus.csv")
Sprvsr.pos_in_t

In [ ]:
## Train
for t in range(Sprvsr.T):

    # # measurement
    print('tip position = ', Sprvsr.pos_in_t[t, :])
    m.move_pos_w_mid(Sprvsr.pos_in_t[t, :], Sprvsr)
    Snsr.measure(2)
    Sprvsr.global_force(Snsr, m, t, plot=True)
    print('measurement force = ', [Sprvsr.Fx, Sprvsr.Fy])
    
    Sprvsr.calc_loss(t, Snsr.norm_force)
    print('loss (normalized) = ', Sprvsr.loss_norm)

    # # update
    Sprvsr.calc_tip_update(m, t)
    m.move_pos_w_mid(Sprvsr.pos_update_in_t[t, :], Sprvsr)

In [ ]:
## Print out results of training

print('update pos in t', Sprvsr.pos_update_in_t)
print('Sprvsr.F_in_t', Sprvsr.F_in_t)
print('Sprvsr.desired_F_in_t', Sprvsr.desired_F_in_t)
plt.plot(range(Sprvsr.T), Sprvsr.loss_norm_in_t)

In [ ]:
## Save as csv

importlib.reload(file_helpers)

T = 8

# path_csv = f"{np.array2string(Sprvsr.init_buckle)}to{np.array2string(Sprvsr.desired_buckle)}.csv"
path_csv = f"1110to0111.csv"
file_helpers.export_training_csv(path_csv, Sprvsr, T=T)

In [ ]:
## plot important training results

importlib.reload(plot_func)
final_t = 13

file_path = "data/measurements/Jan29/1110to0111.csv" 
plot_func.importants_from_file(file_path, final_t=final_t, save=True)

## Check Robot movement

In [ ]:
# m.robot.MoveLin(140, 0, 180, 180, 0, 0)
# points = (220.0, 0.0, 360, 0, 0, 90)
points = (250, 55, -73)  # training metal shims 4piece
# points = (170,  135, -190)
# points = (115, -14, 0)
m.move_pos_w_mid(points, Sprvsr)
# joints = (0, 60, 30, -90, -90, 180)
# m.move_joints(joints)

In [ ]:
m._get_current_pos()
m.current_pos
# m.current_pos[-1]

In [ ]:
Snsr.measure(4)
Sprvsr.global_force(Snsr, m, plot=True)
print('Fx', Sprvsr.Fx)
print('Fy', Sprvsr.Fy)

## Build dataset or determined measurement

In [ ]:
## Measure

importlib.reload(experiments)

# around Jan 20
# x_range = 240
# y_range = 85
# theta_range = 45
# Jan 22
# x_range = 0.25*1000 + 10
# y_range = 0.02*1000 - 14
# theta_range = 0.4*180/np.pi
# Jan 23 metal shims
x_range = 160 + m.pos_origin[0]
y_range = 80 + m.pos_origin[1]
theta_range = np.rad2deg(-0.5)
N = 1
x_y_theta_vec, F_vec = experiments.sweep_measurement_fixed_origami(m, Snsr, Sprvsr, 
                                                                   x_range, y_range,
                                                                   theta_range, N)
print('x_y_theta_vec', x_y_theta_vec)
print('F_vec', F_vec)

In [ ]:
## Save as csv

importlib.reload(file_helpers)
# origin = np.array([154-30, -14, 0])  # 5piece plastic shims
# origin = m.pos_origin
# origin = np.append(origin, 0)
origin = np.array([0, 0, 0])  # for when using sim data
x_y_theta_vec_moved = x_y_theta_vec - origin
print(x_y_theta_vec_moved)
out_path = "buckle[0111]minus.csv"
file_helpers.write_supervisor_dataset(x_y_theta_vec_moved, F_vec, out_path=out_path)

## Detemined set

In [ ]:
# Sprvsr.pos_in_t = np.array([[180, 40, 0], [180, 40, -40], [180, 40, -80], 
#                             [180, 40, -120], [180, 40, -140], [180, 40, -160], 
#                             [180, 40, -180], [180, 40, -200] ])
# Sprvsr.pos_in_t = np.array([[180, 70, -90]])
# Sprvsr.pos_in_t = np.array([[180, 70, -90], [180, 75, -90], [180, 80, -90], 
#                             [180, 85, -90], [180, 90, -90], [180, 95, -90], 
#                             [180, 100, -90], [180, 105, -90]])
# Sprvsr.T = 8
# tri = 1 - np.abs(2 * (np.arange(Sprvsr.T) / (Sprvsr.T - 1)) - 1)   # 0 → 1 → 0
# x_pos = 230 + 10 * tri                   # 170 → 210 → 170
# y_pos = np.linspace(-50, 50, Sprvsr.T)
# Sprvsr.pos_in_t = np.zeros([Sprvsr.T, 3])
# Sprvsr.F_in_t = np.zeros([Sprvsr.T, 2])

# for t in range(Sprvsr.T):
#     Sprvsr.pos = np.array([x_pos[t], y_pos[t], 0])
#     Sprvsr.pos_in_t[t, :] = Sprvsr.pos
#     print('tip position = ', Sprvsr.pos)
#     m.move_pos_w_mid(Sprvsr.pos, Sprvsr)
#     Snsr.measure()
#     Sprvsr.global_force(Snsr, m, plot=True)
#     plt.show()
#     Sprvsr.F_in_t[t, :] = np.array([Sprvsr.Fx, Sprvsr.Fy])
#     print('mean force = ', [Sprvsr.Fx, Sprvsr.Fy])



In [ ]:
plt.errorbar(y_pos, Sprvsr.F_in_t[:,0], yerr=0.01)
plt.errorbar(y_pos, Sprvsr.F_in_t[:,1], yerr=0.01)
plt.ylabel('F [N]')
plt.ylim([-0.015, 0.085])
plt.xlabel('y [mm]')
plt.legend(['Fx', 'Fy'])

In [ ]:
m.disconnect()

## Logging - fixed arm

In [ ]:
experiments.sweep_measurement_fixed_origami(x_range, y_range, theta_range, N, robot, force_sensor)

## Calibrate forces

In [ ]:
importlib.reload(experiments)

weights_gr = [2, 5, 10, 20, 50]
experiments.calibrate_forces_all_axes(m, Snsr, weights_gr)

## Recover robot after error

In [ ]:
m._recover_robot()

## End of day

In [ ]:
# m.robot.MoveLin(40, 0, 230, 180, 0, 0)
m.move_to_sleep_pos()

In [ ]:
m.disconnect()